In [1]:
import torch
import torchtext
import numpy as np
import torchtext
import pandas
import collections
import collections
from torchnlp import *
from torch.optim import lr_scheduler

# Обучение LSTM c посимвольной токенизацией и токенизацией по словам

### Задаём гиперпараметры

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# токенизацией по словам
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
# длина обрабатываемой последовательности токенов
nchars = 100
num_layers=3
test_str_orig = "Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли у него по бокам. Лорд Малфой повернул голову, слишком слабо, чтобы считать это знаком внимания, но всё-таки в направлении профессора Защиты."
test_str_slov = "Беловолосый человек остановился у самого"
test_str = test_str_orig[:nchars]


## Загрузка датасета с токенизацией по словам

In [3]:
def load_dataset_HP(ngrams=1,min_freq=1):
    global vocab, tokenizer
    print("Loading dataset...")
    train_dataset = pandas.read_csv('./dataHPMRM.csv')["text"]
    train_dataset = list(train_dataset)
    print('Building vocab...')
    counter = collections.Counter()
    for line in train_dataset:
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
    vocab = torchtext.vocab.vocab(counter, min_freq=min_freq)
    return train_dataset,vocab
train_dataset,vocab_slov=load_dataset_HP()
vocab_size_slov = len(vocab_slov)

Loading dataset...
Building vocab...


### Посимвольная токенизация

In [4]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for line in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")


Vocabulary size = 167
Encoding of 'a' is 120
Character with code 13 is н


### Функции Кодирования токенов

In [5]:
stoi_hash = {}
def encode(x,voc=None,unk=0,tokenizer=tokenizer):
    global stoi_hash
    v = vocab if voc is None else voc
    if v in stoi_hash.keys():
        stoi = stoi_hash[v]
    else:
        stoi = v.get_stoi()
        stoi_hash[v]=stoi        
    return [stoi.get(s,unk) for s in tokenizer(x)]

In [6]:
## Ф-ии Кодирование токинов по словам
def enc_slov(x):
    return torch.LongTensor(encode(x,voc=vocab_slov,tokenizer=tokenizer))

In [7]:
## Ф-ии Кодирование токинов по символам
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

### ф-ия Бача в эпохе

In [8]:
def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

In [9]:
def get_batch_slov(s,nchars=nchars):
    leng = len(enc_slov(s))
    inps = torch.zeros(leng-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(leng-nchars,nchars,dtype=torch.long,device=device)
    for i in range(leng-nchars):
        inps[i] = enc_slov(s)[i:i+nchars]
        outs[i] = enc_slov(s)[i+1:i+nchars+1]
    return inps,outs

## Однослойная однонаправленая сеть LSTM

In [10]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True,num_layers=1)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

## Многослойная однонаправленая сеть LSTM

In [11]:
class MultilayerLSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim,num_layers):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True,num_layers=num_layers)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

## Генератор следующией послдовательности символов/слов

In [12]:
def generate(net,size=100,start='Сегодня '):
    chars = list(start)
    out, s = net(enc(chars).view(1,-1).to(device))
    for i in range(size):
        nc = torch.argmax(out[0][-1])
        chars.append(vocab.get_itos()[nc])
        out, s = net(nc.view(1,-1),s)
    return ''.join(chars)
        
def generate_slov(net,size=100,start='Сегодня '):
    chars = start
    out, s = net(enc_slov(start).view(1,-1).to(device))
    for i in range(size):
        nc = torch.argmax(out[0][-1])
        chars+= " " + (vocab_slov.get_itos()[nc])
        out, s = net(nc.view(1,-1),s)
    return ''.join(chars)

## Мягкая генерация

In [13]:
def generate_soft(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = list(start)
    out, s = net(enc(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        out_dist = out[0][-1].div(temperature).exp()
        nc = torch.multinomial(out_dist,1)[0]
        chars.append(vocab.get_itos()[nc])
        out, s = net(nc.view(1,-1),s)
    return ''.join(chars)

def generate_soft_slov(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = start
    out, s = net(enc_slov(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        out_dist = out[0][-1].div(temperature).exp()
        nc = torch.multinomial(out_dist,1)[0]
        chars += " " + vocab_slov.get_itos()[nc]
        out, s = net(nc.view(1,-1),s)
    return ''.join(chars)


## Ф-ия обучения

In [14]:
def train(net,train_dataset,val_dataset,num_epochs, optimizer, scheduler, get_batch, generate, slov = False):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for phase in ["T","V"]:
            if phase == 'T':
                net.train()  # Установить модель в режим обучения
                dataset=train_dataset
            else:
                net.eval()   #Установить модель в режим оценки
                dataset=val_dataset
            
            epoch_loss = 0.0    
            for i,x in enumerate(dataset):
                
                # x[0] is class label, x is text
                if slov:
                    leng = len(enc_slov(x))
                else:
                    leng = len(x)
                    
                if leng-nchars<10:
                    continue

                text_in, text_out = get_batch(x,nchars=nchars)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'T'):
                    out,s = net(text_in)
                    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
                    if phase == 'T':
                        loss.backward()
                        optimizer.step()
                    else:
                        # scheduler.step(loss.item())
                        pass
                epoch_loss+=loss.item()
                # if i%100==0:
                #     print(f"{i}/{len(dataset)} Current loss {phase} = {epoch_loss/(i+1)}")
            print(f"Current loss {phase} = {epoch_loss / len(dataset)}")
            if phase == 'V':
                print(generate(net,start=test_str))    

## Обучение однослойной, однонаправленной, посимвольной LSTM

In [15]:
net=LSTMGenerator(vocab_size,64).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [16]:
train(net,train_dataset,train_dataset,10,optimizer, scheduler, get_batch, generate)

Epoch 0/10
Current loss T = 0.8680756190126752
Current loss V = 0.8027476338431492
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерлись старалась старалась старалась старалась старалась старалась старалась старалась старалась старал
Epoch 1/10
Current loss T = 0.7778844116100883
Current loss V = 0.7773748026489273
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерлива, что он не сказала следует следует следует следует следует следует следует следует следует следу
Epoch 2/10
Current loss T = 0.7617493762364437
Current loss V = 0.7696920968429263
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли следует следует следует следует следует следует следует следует следует следует следует следует сл
Epoch 3/10
Current loss T = 0.7541661817350441
Current loss V = 0.7658954659176807
Беловолосый человек остановился у самого выхода из ложи. Пожилой 

In [17]:
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start=test_str_slov,temperature=i ,enc=enc)}\n")

--- Temperature = 0.3
Беловолосый человек остановился у самого подобной совсем на себя с самом деле в произнесла Гарри Поттер, что она соболилась под кого в старая мои стал на него в произнёс с него следует в самом говорить произнёс в собственный мальчикам свои света, и на следующему и разруданной старая в подобный стороны полагал Гарри Поттер, которое следует

--- Temperature = 0.8
Беловолосый человек остановился у самого его больше нарушильны, я о твоево сила следует быть была мысль, когда и для моей с «возможно удастром акриковой совершен, что моё Возможет нвавного оцинания. Гарри меня арре Хогвартса песса, кразил, что это Макфинной, как может, а следующим удивовать следующи, с нему не держать застались, ставила н

--- Temperature = 1.0
Беловолосый человек остановился у самого обходит я когда, и. Я зататываться пока меня подолах, ямела, но, сле офипЫ. Я не посмкю и — не волшебн решил, как этого убилось, что Гермиона, человечный матоватит, — на так местом, вы того макт она не в того,

## Обучение многослойной, однонаправленной, посимвольной LSTM

In [18]:
net=MultilayerLSTMGenerator(vocab_size,64,num_layers=num_layers).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [19]:
train(net,train_dataset,train_dataset,10,optimizer, scheduler, get_batch, generate)

Epoch 0/10
Current loss T = 0.9168755651688021
Current loss V = 0.7913404158649692
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерлина старался стороны стараясь стараясь стараясь стараясь стараясь стараясь стараясь стараясь стараяс
Epoch 1/10
Current loss T = 0.7617506417925591
Current loss V = 0.760027859768543
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерли, что ты следует просто стараясь с подобный стараясь с подобный стараясь с подобный стараясь с подо
Epoch 2/10
Current loss T = 0.7411081624397015
Current loss V = 0.7464274825756004
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерливалась с следующий старая старая старая старая старая старая старая старая старая старая старая ста
Epoch 3/10
Current loss T = 0.7317723418076555
Current loss V = 0.7378367579950826
Беловолосый человек остановился у самого выхода из ложи. Пожилой ч

In [20]:
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start=test_str,temperature=i ,enc=enc)}\n")

--- Temperature = 0.3
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерла и сторона от силы, не волшебника, не старалась, что в заклинание, словно придумать бы устроила на случае о старался на подобного магический своих не смогу в сторона с дементора самого сторона происпользовать так и подобный какой-то придумала своей над собственный страни, и ты не смог подумать в св

--- Temperature = 0.8
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерлисшего мобей о безопасти и усути Гермиона, я в том, что ты проволял, на волшеблимента, которой хотя? И на самом деле на этому в своему для хотя не я сделать в раз, как этот Люпичении, как Гарри, чтобы Гарри, чтобы довольного Светов профессора не возражать как-то каждые существился, в всё это казавле

--- Temperature = 1.0
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерлась будет ты себен защищают, н

## Обучение однослойной, однонаправленной, пословной LSTM

In [15]:
nchars = 10
vocab_size = len(vocab_slov)

In [16]:
net=LSTMGenerator(vocab_size,64).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [17]:
train(net,train_dataset,train_dataset,10,optimizer, scheduler, get_batch_slov, generate_slov, slov = True)

Epoch 0/10
Current loss T = 3.2940392755369468
Current loss V = 2.850608828217078
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерл , — что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь сделать , что ты можешь
Epoch 1/10
Current loss T = 2.7070711377160372
Current loss V = 2.543373460888091
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерл , и в горле . — ты не можешь сделать , — сказала гермиона . — это не было . — сказала гермиона . — это не было . — сказала гермиона . — это не было . — сказала 

In [46]:
def generate_soft_slov(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = start
    out, s = net(enc_slov(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        print(out.shape)
        out_dist = out[0][-1].div(temperature).exp()
        print(sum(out[0][-1].div(temperature)))
        nc = torch.multinomial(out_dist,1)[0]
        chars += " " + vocab_slov.get_itos()[nc]
        out, s = net(nc.view(1,-1),s)
    return ''.join(chars)

In [52]:
print(f"Результат\n{generate_slov(net,size=300,start=test_str_slov)}\n")

Результат
Беловолосый человек остановился у самого , и в самом деле , — сказала гермиона . — я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказал гарри , — я могу сделать , но я не могу сделать , но я не могу . я не могу сделать , — сказ

## Обучение многослойной, однонаправленной, пословной LSTM

In [53]:
net=MultilayerLSTMGenerator(vocab_size,64,num_layers=num_layers).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [54]:
train(net,train_dataset,train_dataset,10,optimizer, scheduler, get_batch_slov, generate_slov, slov = True)

Epoch 0/10
Current loss T = 3.4826228385708102
Current loss V = 3.231301177563483
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерл , — сказала гермиона . — я не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не
Epoch 1/10
Current loss T = 3.0099347712319275
Current loss V = 3.2114101676480513
Беловолосый человек остановился у самого выхода из ложи. Пожилой человек и человек со шрамами замерл , — сказала гермиона . — я не могу бы бы бы бы , что ты не можешь бы бы бы бы бы бы бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы , что ты не могу бы бы бы 

In [55]:
print(f"Результат\n{generate_slov(net,size=300,start=test_str_slov)}\n")

Результат
Беловолосый человек остановился у самого гарри , — сказала гермиона . — я не могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если ты не могу бы рассказать о то , что мне не было изменять этого гласила , что я могу предначертано подчеркнуть такого , если 